In [1]:
# from quantitative_analysis import fetch_stock_data, calculate_technical_indicators
import plotly.express as px
import yfinance as yf
import pandas as pd
import numpy as np

# Structured output

In [20]:
import ollama
from pydantic import BaseModel
from typing import Dict, List

In [15]:
class TickerResponse(BaseModel):
    name: str
    ticker: str

class TickerEvaluation(BaseModel):
    they_are_same:bool

In [4]:
model_list = list()

for model in ollama.list().models:
    model_list.append(model.model)

model_list

['llama3.2:3b',
 'llama3.2:1b',
 'PetrosStav/gemma3-tools:4b',
 'gemma3:4b',
 'gemma3:1b',
 'gemma2:2b',
 'gemma2:latest',
 'llama3:latest',
 'llama3:8b']

In [5]:
def fetch_stock_data(ticker:str) -> pd.DataFrame:
    """
    Fetch the stock data of a ticker from yahoo finance.

    Args:
        ticker: A stock ticker string symbol
    
    Returns:
        Stock Data
    """
    stock = yf.download(ticker, period="18mo")
    stock.columns = stock.columns.droplevel(1) # Make the multi-index columns to a single level column; compatible with plotly
    return stock

In [6]:
user_query = 'The company that makes CUDA supported GPU'
# user_query = 'The biggest company in the US'
# user_query = 'The biggest SNS company'

In [7]:
response = ollama.chat(
    model = 'gemma3:4b', 
    messages = [
        {"role": "user",
         "content": f"What is the company name and finance ticker for this company? {user_query}"}
    ],
    format = TickerResponse.model_json_schema()
)

In [8]:
llm_query = TickerResponse.model_validate_json(response.message.content)
name = llm_query.name
ticker = llm_query.ticker

In [19]:
name_from_ticker = yf.Ticker(ticker).info['longName']
name_from_ticker

'NVIDIA Corporation'

In [23]:
# for testing
name_from_ticker = 'APPLE'

In [24]:
evaluation = ollama.chat(
    model = 'gemma3:4b',
    messages = [
        {"role": "user",
         "content": f"Do these two mean the same company? {name} and {name_from_ticker}"}
    ],
    format = TickerEvaluation.model_json_schema()
)

In [25]:
llm_evaluation = TickerEvaluation.model_validate_json(evaluation.message.content)
llm_evaluation.they_are_same

False

In [21]:
data = fetch_stock_data(ticker)

[*********************100%***********************]  1 of 1 completed


In [22]:
data

Price,Close,High,Low,Open,Volume
Date,,,,,
2023-09-20,42.219433,43.882664,42.203441,43.579802,367108000
2023-09-21,40.997997,42.080496,40.961016,41.563737,450736000
2023-09-22,41.590721,42.095488,41.211895,41.552737,479236000
2023-09-25,42.202442,42.516296,41.157924,41.571734,419091000
2023-09-26,41.891582,42.800162,41.635701,41.981541,402282000
...,...,...,...,...,...
2025-03-13,115.580002,117.760002,113.790001,117.029999,299033100
2025-03-14,121.669998,121.879997,118.150002,118.610001,277593500
2025-03-17,119.529999,122.889999,118.029999,122.739998,255501500


In [23]:
def calculate_SMA(stock:pd.DataFrame, window:int) -> pd.DataFrame:
    """
    Calculate the SMA (Top Hat Filter) with a given window size of the past 6 months.
    """
    # Get the six_month_ago date: The first date shown on the graph
    six_month_ago = stock.index[-1] - pd.DateOffset(months = 6)
    six_month_ago = stock.index[stock.index > six_month_ago].min().strftime("%Y-%m-%d")

    # Fetch the right amount of data for SMA calculation
    six_month_ago_index = np.where(stock.index == six_month_ago)[0][0]
    stock_for_SMA = stock.iloc[six_month_ago_index-window:].copy()

    SMA = stock_for_SMA["Close"].rolling(window=window).mean()
    stock[f'SMA {window}'] = SMA
    return stock

In [24]:

stock = fetch_stock_data(ticker)
stock = calculate_SMA(stock, 50)
stock = calculate_SMA(stock, 200)

[*********************100%***********************]  1 of 1 completed


In [25]:
# Plot stock price
six_month_ago = stock.index[-1] - pd.DateOffset(months = 6)
six_month_ago = stock.index[stock.index > six_month_ago].min().strftime("%Y-%m-%d")
stock_to_show = stock.loc[stock.index > six_month_ago]
fig = px.line(stock_to_show, 
              x=stock_to_show.index, 
              y=["Close", "SMA 200", "SMA 50"], 
              title=f"{company_name} Stock Price & SMAs")

fig.show()

In [14]:
ticker = 'META'

print('sector', yf.Ticker(ticker).info['sector'])
print('industry', yf.Ticker(ticker).info['industry'])

sector Communication Services
industry Internet Content & Information


In [15]:
yf.Ticker(ticker).info

{'address1': '1 Meta Way',
 'city': 'Menlo Park',
 'state': 'CA',
 'zip': '94025',
 'country': 'United States',
 'phone': '650 543 4800',
 'website': 'https://investor.atmeta.com',
 'industry': 'Internet Content & Information',
 'industryKey': 'internet-content-information',
 'industryDisp': 'Internet Content & Information',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'Meta Platforms, Inc. engages in the development of products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality and mixed reality headsets, augmented reality, and wearables worldwide. It operates through two segments, Family of Apps (FoA) and Reality Labs (RL). The FoA segment offers Facebook, which enables people to build community through feed, reels, stories, groups, marketplace, and other; Instagram that brings people closer through instagram feed, s

In [7]:
from duckduckgo_search import DDGS

def fetch_financial_news(name):
    results = DDGS().news(keywords=f"{name} financial report")
    return results

In [97]:
result = fetch_financial_news('Nvidia')

In [98]:
result

[{'date': '2025-03-20T11:39:44+00:00',
  'title': 'Nvidia Considers Massive US Supply Chain Expansion, Report Says',
  'body': 'KEY TAKEAWAYS Nvidia could spend "several hundred billion" dollars on chips and other electronics to be made in the U.S. over the next four years, CEO Jensen Huang said in an interview with the Financial Times.',
  'url': 'https://www.msn.com/en-us/money/markets/nvidia-considers-massive-us-supply-chain-expansion-report-says/ar-AA1BiXmf',
  'image': 'https://img-s-msn-com.akamaized.net/tenant/amp/entityid/AA1p4RLj.img?w=3426&h=2396&m=4&q=60',
  'source': 'Investopedia'},
 {'date': '2025-03-20T09:53:23+00:00',
  'title': 'Nvidia to invest hundreds of billions in US made chips, electronics - report',
  'body': 'Nvidia (NASDAQ:NVDA) plans to spend hundreds of billion dollars on chips and other electronics made in the U.S. over the next four years, the Financial Times reported. "Overall, we will procure, over the course of the next four years,',
  'url': 'https://w

In [99]:
articles = ''

articles = articles.join([r['body'] for r in result])

articles

'KEY TAKEAWAYS Nvidia could spend "several hundred billion" dollars on chips and other electronics to be made in the U.S. over the next four years, CEO Jensen Huang said in an interview with the Financial Times.Nvidia (NASDAQ:NVDA) plans to spend hundreds of billion dollars on chips and other electronics made in the U.S. over the next four years, the Financial Times reported. "Overall, we will procure, over the course of the next four years,Nvidia plans to invest hundreds of billions of dollars in U.S.-made chips and electronics over the next four yearsIn its fourth fiscal quarter ending last month, Nvidia reported $39.3 billion in revenue, $0.89 adjusted earnings per share and $22.1 billion of net income, equating to year-over-year revenue growth of 78% and profit growth of 71%.Nvidia Corp. aims to spend several hundred billion dollars to procure US-made chips and electronics over the next four years, the Financial Times reported.NVDA stock rose after hours as Nvidia\'s earnings resul

In [100]:
[r['body'] for r in result]

['KEY TAKEAWAYS Nvidia could spend "several hundred billion" dollars on chips and other electronics to be made in the U.S. over the next four years, CEO Jensen Huang said in an interview with the Financial Times.',
 'Nvidia (NASDAQ:NVDA) plans to spend hundreds of billion dollars on chips and other electronics made in the U.S. over the next four years, the Financial Times reported. "Overall, we will procure, over the course of the next four years,',
 'Nvidia plans to invest hundreds of billions of dollars in U.S.-made chips and electronics over the next four years',
 'In its fourth fiscal quarter ending last month, Nvidia reported $39.3 billion in revenue, $0.89 adjusted earnings per share and $22.1 billion of net income, equating to year-over-year revenue growth of 78% and profit growth of 71%.',
 'Nvidia Corp. aims to spend several hundred billion dollars to procure US-made chips and electronics over the next four years, the Financial Times reported.',
 "NVDA stock rose after hours a

In [101]:
import ollama
from typing import Literal

In [102]:
class NewsAnalysisStructure(BaseModel):
    events: list[str]
    strengths: list[str]
    weaknesses: list[str]
    grade: Literal['Strongly Recommend', 'Recommend', 'Neutral', 'Risky', 'Highly Risky']
    comment: str

In [112]:
system_prompt = """
    You are a helpful financial analyst. Your task is to read and analyze articles about a company,
    and extract three key events, strengths and weaknesses based on the article and your analysis for buy or sell decision of the company's stock or index fund's ETF.
    Grade the stock or ETF based on your analysis and provide a very short single-sentence comment.
    Highlight the keywords with compliance of markdown format if necessary.
"""

In [113]:
response = ollama.chat(
    model = 'gemma3:4b',
    messages =[
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': articles
        }
    ], 
    format = NewsAnalysisStructure.model_json_schema()
)

In [114]:
llm_news_analysis = NewsAnalysisStructure.model_validate_json(response.message.content)

In [115]:
events = llm_news_analysis.events
weaknesses = llm_news_analysis.weaknesses
strengths = llm_news_analysis.strengths
grade = llm_news_analysis.grade
comment = llm_news_analysis.comment

In [116]:
for strength in strengths:
    print(strength)
    print()

Strong Revenue Growth: Nvidia reported 78% revenue growth and 71% profit growth year-over-year, demonstrating robust demand for its products, particularly Blackwell chips.

Market Dominance Strategy: Nvidia is actively pursuing strategies to solidify its market leadership against competitors like Broadcom and AMD.

Strategic Investments: The company’s significant investment plans signal confidence in its long-term growth prospects and commitment to innovation.



In [117]:
for weakness in weaknesses:
    print(weakness)
    print()

External Risks: Concerns regarding U.S. tariffs and competing AI products in China pose potential headwinds.

Recalls and Safety Issues: Tesla's CyberTruck recall highlights potential risks associated with product safety and execution.

Reliance on AI Trends: Nvidia’s success is heavily tied to the continued growth and evolution of the AI market, which is subject to rapid change and competition.



In [118]:
print(grade)

Strongly Recommend


In [119]:
print(comment)

Despite external uncertainties, Nvidia’s aggressive investment strategy and demonstrated market leadership position it as a compelling buy for long-term investors.
